In [1]:
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore, SPARQLStore
from rdflib import Graph, Namespace, URIRef

from xml.dom.minidom import parse
import xml.etree.ElementTree as etree
from Marc_to_Bibframe.Marc.marcWork import MarcWork
from Marc_to_Bibframe.Marc.marcInstance import MarcInstance
from Marc_to_Bibframe.Marc.marcItems import MarcItems
from Marc_to_Bibframe.Work.work import Work
from Marc_to_Bibframe.Instance.instance import Instance

import pysolr
#from Solr.solr import create_doc
from Jena_cli.jena import UpadateJena

DELETE ALL GRAPHS

In [2]:
store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/bibframe/update')
query_endpoint = 'http://localhost:3030/bibframe/query'
update_endpoint = 'http://localhost:3030/bibframe/update'
store.open((query_endpoint, update_endpoint))

In [5]:
#Delete all Named Graph

d = """DELETE { graph ?g { ?s ?p ?o } } 

WHERE {

graph ?g {?s ?p ?o.}

}"""

store.update(d)

SOLR

In [10]:
#SOLR
solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":6,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-1"}},\n  "status":"OK"}\n'

In [2]:
def UpadateSolr(path_marc, count, shelf):
    
    marc_file = parse(path_marc)
    records = marc_file.getElementsByTagName('record')

    docs = list()
    for record in records:
        
        marcxml = etree.fromstring(record.toxml())
        workMarc = MarcWork(marcxml)
        instanceMarc = MarcInstance(marcxml)
        itemsMarc = MarcItems(marcxml)
        print(count)
        print(workMarc.Title().get('title'))

        #SOLR
        doc = create_doc(workMarc, instanceMarc, itemsMarc, 'Livro', shelf, count)
        docs.append(doc)
        
        count += 1
    return docs, count

In [ ]:
count = 378
e = '3'
p = '5'
out = f'E{e}/P{p}'
path_marc = f'Input/{out}/koha.xml'

docs, _ = UpadateSolr(path_marc, count, f"E{e}.P{p}")

In [49]:
#solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.add(docs, commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":348}}\n'

IMPORT LOTE

In [ ]:
count = 1
solr_docs = list()
for e in range(1, 3):
    for p in range(1,6):
        path_marc = f'Input/E{e}/P{p}/koha.xml'
        docs, count  = UpadateSolr(path_marc, count, f"E{e}.P{p}")
        #solr_docs.append(docs)
        solr_docs = solr_docs + docs
        #count += 1
        #print(count)

In [35]:
solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.add(solr_docs, commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":1736}}\n'

JENA

In [ ]:
count = 1
for i in range(1,3):
    e = i
    for p in range(1,6):
        out = f'E{e}/P{p}'
        path_marc = f'Input/{out}/koha.xml'
        shelf = f"E{e}.P{p}"
        count = UpadateJena(path_marc, count, shelf, out)


In [ ]:
e = 2
count = 106
for p in range(1,6):
    out = f'E{e}/P{p}'
    path_marc = f'Input/{out}/koha.xml'
    count = UpadateJena(path_marc, count, shelf, out)

In [7]:
out = f'E1/P1'
path_marc = f'Input/{out}/koha.xml'
marc_file = parse(path_marc)
records = marc_file.getElementsByTagName('record')
record = records[0]
marcxml = etree.fromstring(record.toxml())
workMarc = MarcWork(marcxml)
instanceMarc = MarcInstance(marcxml)
itemsMarc = MarcItems(marcxml) 

In [3]:
BFwork = URIRef(f"http://bibliokeia.com/bibframe/work/1") 
BFinstance = URIRef(f"http://bibliokeia.com/bibframe/instance/1")
work = Work(1, workMarc, BFwork, BFinstance)

In [5]:
work.serialize('TESTE.ttl', format='turtle')

<Graph identifier=http://bibliokeia.com/bibframe/work/1 (<class 'rdflib.graph.Graph'>)>

In [6]:
from Solr.create_doc import Create_doc

In [9]:
doc = Create_doc(workMarc, instanceMarc, itemsMarc, 'Livro', 'P1.E1', 1)